# FL-EHDS — HealthcareCNN Imaging Comparison

Confronto HealthcareCNN (~500K params) vs ResNet-18 (11.2M params) su dataset imaging.

- **27 esperimenti**: 3 algos (FedAvg, Ditto, HPFL) × 3 datasets × 3 seeds
- **Tempo stimato**: ~1.5-2.5 ore su GPU T4/A100
- **Checkpoint**: salvataggio automatico su Google Drive ogni esperimento

**IMPORTANTE**: Runtime → Change runtime type → **GPU (T4)**

## 1. Setup Environment

In [ ]:
# Mount Google Drive for persistent checkpoint storage
from google.colab import drive
drive.mount('/content/drive')

import os
DRIVE_OUTPUT = '/content/drive/MyDrive/FL-EHDS-FLICS2026/colab_results'
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
print(f'Drive output: {DRIVE_OUTPUT}')

In [ ]:
# Check GPU
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    props = torch.cuda.get_device_properties(0)
    mem = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
    print(f'Memory: {mem / 1e9:.1f} GB')
else:
    print('ATTENZIONE: Nessuna GPU! Vai su Runtime -> Change runtime type -> GPU')

In [ ]:
# Clone repository
!git clone https://github.com/FabioLiberti/FL-EHDS-FLICS2026.git /content/FL-EHDS-FLICS2026
%cd /content/FL-EHDS-FLICS2026/fl-ehds-framework
!pip install -q opacus>=1.4.0 scikit-learn scipy tqdm rich pydantic pyyaml

## 2. Download Datasets (da Kaggle)

In [ ]:
# Setup Kaggle API
!pip install -q kagglehub

import os
os.environ['KAGGLE_API_TOKEN'] = 'KGAT_edd561c1bc682c9ad06930bacd164431'

import kagglehub
print(f'kagglehub version: {kagglehub.__version__}')
print('Kaggle auth ready')

In [ ]:
%%time
# Download Chest X-Ray Pneumonia (~2.3 GB)
import kagglehub, shutil, os

cache_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print(f'Downloaded to cache: {cache_path}')

os.makedirs('data/chest_xray', exist_ok=True)
for item in ['train', 'test', 'val']:
    src = os.path.join(cache_path, 'chest_xray', item)
    if not os.path.exists(src):
        src = os.path.join(cache_path, item)
    dst = f'data/chest_xray/{item}'
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copytree(src, dst)
        print(f'  Copied {item}')

shutil.rmtree('data/chest_xray/__MACOSX', ignore_errors=True)
print('Chest X-Ray ready:')
!find data/chest_xray -name '*.jpeg' -o -name '*.jpg' -o -name '*.png' | wc -l

In [ ]:
%%time
# Download Skin Cancer (~325 MB)
cache_path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")
print(f'Downloaded to cache: {cache_path}')

dst = 'data/Skin Cancer'
if not os.path.exists(dst):
    shutil.copytree(cache_path, dst)

print('Skin Cancer ready:')
!find "data/Skin Cancer" -name '*.jpg' -o -name '*.jpeg' -o -name '*.png' | wc -l

In [ ]:
%%time
# Download Brain Tumor (~250 MB)
import glob

cache_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print(f'Downloaded to cache: {cache_path}')

os.makedirs('data/Brain_Tumor', exist_ok=True)
for root, dirs, files in os.walk(cache_path):
    for d in dirs:
        d_lower = d.lower()
        if d_lower in ['glioma', 'meningioma', 'pituitary', 'notumor', 'no_tumor', 'healthy']:
            target = 'healthy' if d_lower in ['notumor', 'no_tumor'] else d_lower
            src = os.path.join(root, d)
            dst_dir = f'data/Brain_Tumor/{target}'
            if not os.path.exists(dst_dir):
                shutil.copytree(src, dst_dir)
            else:
                for f in os.listdir(src):
                    src_f = os.path.join(src, f)
                    dst_f = os.path.join(dst_dir, f)
                    if os.path.isfile(src_f) and not os.path.exists(dst_f):
                        shutil.copy2(src_f, dst_f)

print('Brain Tumor ready:')
!find data/Brain_Tumor -name '*.jpg' -o -name '*.jpeg' -o -name '*.png' | wc -l
!ls data/Brain_Tumor/

In [ ]:
# Verify all datasets + remove macOS ._ files
import subprocess
subprocess.run(['find', 'data/', '-name', '._*', '-delete'], capture_output=True)

print('=== Dataset Summary ===')
for ds_name, ds_path in [('Chest X-Ray', 'data/chest_xray'),
                          ('Skin Cancer', 'data/Skin Cancer'),
                          ('Brain Tumor', 'data/Brain_Tumor')]:
    count = sum(1 for _ in glob.iglob(f'{ds_path}/**/*.*', recursive=True)
                if _.lower().endswith(('.jpg', '.jpeg', '.png')))
    subdirs = [d for d in os.listdir(ds_path) if os.path.isdir(os.path.join(ds_path, d))]
    print(f'  {ds_name:15s}: {count:5d} images, classes: {subdirs}')

## 3. Run HealthcareCNN Experiments

27 esperimenti con checkpoint su Google Drive.
Se la sessione si disconnette, ri-esegui questa cella — riprende automaticamente.

In [ ]:
%%time
import sys, json, time, gc, traceback, shutil
from pathlib import Path
from datetime import datetime
import numpy as np, torch

FRAMEWORK_DIR = Path('/content/FL-EHDS-FLICS2026/fl-ehds-framework')
sys.path.insert(0, str(FRAMEWORK_DIR))
from terminal.fl_trainer import ImageFederatedTrainer, _detect_device

ALGORITHMS = ["FedAvg", "Ditto", "HPFL"]
SEEDS = [42, 123, 456]
DATASETS = {
    "chest_xray": {"data_dir": str(FRAMEWORK_DIR / "data" / "chest_xray"), "num_classes": 2},
    "Brain_Tumor": {"data_dir": str(FRAMEWORK_DIR / "data" / "Brain_Tumor"), "num_classes": 4},
    "Skin_Cancer": {"data_dir": str(FRAMEWORK_DIR / "data" / "Skin Cancer"), "num_classes": 2},
}
CONFIG = dict(
    num_clients=5, num_rounds=20, local_epochs=2, batch_size=32,
    learning_rate=0.001, model_type="cnn",
    is_iid=False, alpha=0.5, freeze_backbone=False, freeze_level=0,
    use_fedbn=True, use_class_weights=True, use_amp=True, mu=0.1,
)
EARLY_STOPPING = dict(enabled=True, patience=4, min_delta=0.003, min_rounds=8, metric="accuracy")

# Checkpoint su Google Drive (persiste tra sessioni)
DRIVE_OUTPUT = Path('/content/drive/MyDrive/FL-EHDS-FLICS2026/colab_results')
DRIVE_OUTPUT.mkdir(parents=True, exist_ok=True)
CKPT_FILE = DRIVE_OUTPUT / 'checkpoint_imaging_cnn.json'

def load_checkpoint():
    if CKPT_FILE.exists():
        with open(CKPT_FILE) as f:
            return json.load(f)
    return {"completed": {}, "meta": {"started": str(datetime.now()), "model": "HealthcareCNN"}}

def save_checkpoint(ckpt):
    tmp = CKPT_FILE.with_suffix('.tmp')
    with open(tmp, 'w') as f:
        json.dump(ckpt, f, indent=2, default=str)
    tmp.rename(CKPT_FILE)

device = _detect_device()
ckpt = load_checkpoint()
experiments = [(ds, algo, seed) for ds in DATASETS for algo in ALGORITHMS for seed in SEEDS]
total = len(experiments)
done = len([k for k, v in ckpt["completed"].items() if "error" not in v])
print(f"=== HealthcareCNN Imaging Comparison ===")
print(f"Total: {total}, Already done: {done}, Remaining: {total - done}")
print(f"Device: {device}")
print(f"Checkpoint: {CKPT_FILE}\n")

for i, (ds, algo, seed) in enumerate(experiments):
    key = f"{ds}_{algo}_s{seed}"
    if key in ckpt["completed"] and "error" not in ckpt["completed"][key]:
        continue
    print(f"\n[{done+1}/{total}] {key}")
    t0 = time.time()
    try:
        ds_info = DATASETS[ds]
        cfg = {**CONFIG, "num_classes": ds_info["num_classes"]}
        if ds == "Brain_Tumor":
            cfg["learning_rate"] = 0.0005
        np.random.seed(seed); torch.manual_seed(seed)
        trainer = ImageFederatedTrainer(
            data_dir=ds_info["data_dir"], algorithm=algo, seed=seed,
            early_stopping_config=EARLY_STOPPING, **cfg
        )
        result = trainer.train()
        elapsed = time.time() - t0
        ckpt["completed"][key] = {
            "accuracy": result.get("final_accuracy", 0),
            "best_accuracy": result.get("best_accuracy", 0),
            "rounds": result.get("rounds_completed", 0),
            "time_s": round(elapsed, 1),
            "model": "HealthcareCNN",
            "algorithm": algo,
            "dataset": ds,
            "seed": seed,
        }
        done += 1
        print(f"  Done: acc={ckpt['completed'][key]['best_accuracy']:.4f}, {elapsed:.0f}s")
        del trainer; gc.collect(); torch.cuda.empty_cache()
    except Exception as e:
        ckpt["completed"][key] = {"error": str(e), "traceback": traceback.format_exc()}
        print(f"  ERROR: {e}")
    save_checkpoint(ckpt)

print(f"\n=== COMPLETED: {done}/{total} ===")
print(f"\n{'Dataset':<15} {'Algorithm':<10} {'Acc (mean +/- std)'}")
print('-' * 45)
for ds in DATASETS:
    for algo in ALGORITHMS:
        accs = [ckpt['completed'].get(f'{ds}_{algo}_s{s}', {}).get('best_accuracy', 0) for s in SEEDS]
        accs = [a for a in accs if a > 0]
        if accs:
            print(f"{ds:<15} {algo:<10} {np.mean(accs)*100:.1f} +/- {np.std(accs)*100:.1f}%")

## 4. Download Results

In [ ]:
# Download checkpoint (gia salvato su Drive, ma anche scaricabile)
from google.colab import files
ckpt_path = '/content/drive/MyDrive/FL-EHDS-FLICS2026/colab_results/checkpoint_imaging_cnn.json'
if os.path.exists(ckpt_path):
    files.download(ckpt_path)
    print('Downloaded!')
else:
    print('No checkpoint found.')